In [59]:
import requests
import json
import pandas as pd
import time
import random

In [60]:
from datetime import datetime, timedelta

# Current time in UTC
now = datetime.utcnow()

# Calculate the start time and end time
start_time = now.strftime("%Y-%m-%dT%H:%M:%SZ")  # Format as ISO 8601
end_time = (now + timedelta(weeks=1)).strftime("%Y-%m-%dT%H:%M:%SZ")  # 3 weeks later

# Construct the URL
print(start_time)
print(end_time)

2025-01-21T18:05:01Z
2025-01-28T18:05:01Z


In [61]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store each batch of matches data
all_matches = []

# Loop over drillDownTagIds ranges in steps of 900
for start in range(0, 14000, 200):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        "startTimeFrom=2025-01-21T16:00:00Z&"
        "startTimeTo=2025-02-03T22:59:59Z&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "categoryCode=tennis&"  # Specify the desired category
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        for event in data.get('data', {}).get('events', []):
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)
            
            matches.append(match)

        # Add the batch of matches to the all_matches list
        all_matches.extend(matches)

        # Sleep for a random duration between 1 and 4 seconds, with variability of 0.1 ms
        time.sleep(random.uniform(1, 2.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 200}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the resulting DataFrame
print(final_matches_df)

An error occurred for drilldown ID range 0–200: 'NoneType' object is not iterable
An error occurred for drilldown ID range 400–600: 'NoneType' object is not iterable
     event_id                             match_name            start_time  \
0     6629513                                   Lang  2025-01-21T20:00:00Z   
1     6591064         Derby County vs Sunderland AFC  2025-01-21T19:45:00Z   
2     6591065  Middlesbrough vs West Bromwich Albion  2025-01-21T19:45:00Z   
3     6591070       Swansea City vs Sheffield United  2025-01-21T19:45:00Z   
4     6591074           Watford vs Preston North End  2025-01-21T19:45:00Z   
...       ...                                    ...                   ...   
2021  6638317           Jason Banks vs Alex Marshall  2025-01-23T19:30:00Z   
2022  6641087         Julie Forrest vs Nicole Rogers  2025-01-22T15:00:00Z   
2023  6638319          Martin Heitzman vs Mark Dawes  2025-01-23T10:00:00Z   
2024  6638321         Michael Stepney vs Les Gillett  

In [62]:
len(final_matches_df)

2026

In [82]:
final_matches_df['event_id'].nunique()

1367

In [76]:
final_matches_df[final_matches_df['match_name'].str.contains('PSV')]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
330,6285726,PSV vs NAC,2025-01-25T17:45:00Z,PSV,NAC,Nederland Eredivisie,Nederland,Voetbal,1.11,20.00
338,6351955,NEC vs PSV,2025-02-01T20:00:00Z,NEC,PSV,Nederland Eredivisie,Nederland,Voetbal,8.25,1.26
349,6626894,Jong PSV vs VVV Venlo,2025-01-27T19:00:00Z,Jong PSV,VVV Venlo,Nederland Eerste Divisie,Nederland,Voetbal,2.37,2.65
968,6248641,FK Crvena Zvezda vs PSV,2025-01-21T20:00:00Z,FK Crvena Zvezda,PSV,UEFA Champions League,Europees,Voetbal,4.25,1.74
1030,6631121,PSV & Barcelona,2025-01-21T20:00:00Z,None,None,Boost van de Dag,Specials,TOTO Specials,NaN,NaN


In [83]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 30

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        # Get a batch of event_ids
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        
        # Concatenate event_ids for the URL
        event_ids_str = ','.join(map(str, batch_ids))
        
        # Define the URL for the current batch
        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&"
            "includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"
        )
        
        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }
        
        # Send request
        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Initialize lists to store the data for this batch
        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        # Process the JSON response for each event
        for event in json_data['data']['events']:  # Iterate over each event
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:  # Iterate over each market in the event
                market_name = market['name']
                
                for outcome in market['outcomes']:  # Iterate over outcomes for each market
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']
                    
                    # Append captured values to the respective lists
                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        # Create a temporary DataFrame for this batch and append it to all_data
        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })
        
        all_data.append(temp_df)
        
        # Wait between .. and .. seconds
        time.sleep(random.uniform(0.02, 1.81))
    except:
        None

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

      event_id                                 Event Name      Market Name  \
0      6591063          Blackburn Rovers vs Coventry City      Draw No Bet   
1      6591063          Blackburn Rovers vs Coventry City      Draw No Bet   
2      6591063          Blackburn Rovers vs Coventry City     Dubbele Kans   
3      6591063          Blackburn Rovers vs Coventry City     Dubbele Kans   
4      6591063          Blackburn Rovers vs Coventry City     Dubbele Kans   
...        ...                                        ...              ...   
97199  6513026  Said Nurmagomedov vs Vinicius de Oliveira  Winnaar Gevecht   
97200  6513027  Sergei Pavlovich vs Jairzinho Rozenstruik  Winnaar Gevecht   
97201  6513027  Sergei Pavlovich vs Jairzinho Rozenstruik  Winnaar Gevecht   
97202  6308709      Sharaputdin Magomedov vs Michael Page  Winnaar Gevecht   
97203  6308709      Sharaputdin Magomedov vs Michael Page  Winnaar Gevecht   

                            Outcome Name  Odds (Decimal)  Price

In [84]:
final_df[final_df['Event Name'].str.contains('PSV')]['Event Name'].unique()

array(['PSV vs NAC', 'Jong PSV vs VVV Venlo', 'NEC vs PSV',
       'FK Crvena Zvezda vs PSV', 'PSV & Barcelona'], dtype=object)

In [85]:
# Perform the left join on 'event_id' to get 'sport' and 'group_name' from events_df
final_df = final_df.merge(final_matches_df[['event_id', 'sport', 'competition']], on='event_id', how='left').drop_duplicates()
# Remove columns that end with '_x'
final_df = final_df[[col for col in final_df.columns if not col.endswith('_x')]]

# Rename columns to remove '_y'
final_df.columns = final_df.columns.str.replace('_y', '', regex=False)

In [86]:
# Replace 'A' with 2 and 'H' with 1 in the outcome_subtype column
final_df['Outcome SubType'] = final_df['Outcome SubType'].replace({'A': '2', 'H': '1'})

# Display the updated final_df
print(final_df)

       event_id                                 Event Name      Market Name  \
0       6591063          Blackburn Rovers vs Coventry City      Draw No Bet   
1       6591063          Blackburn Rovers vs Coventry City      Draw No Bet   
2       6591063          Blackburn Rovers vs Coventry City     Dubbele Kans   
3       6591063          Blackburn Rovers vs Coventry City     Dubbele Kans   
4       6591063          Blackburn Rovers vs Coventry City     Dubbele Kans   
...         ...                                        ...              ...   
136258  6513029           Shamil Gaziev vs Thomas Petersen  Winnaar Gevecht   
136263  6513027  Sergei Pavlovich vs Jairzinho Rozenstruik  Winnaar Gevecht   
136264  6513027  Sergei Pavlovich vs Jairzinho Rozenstruik  Winnaar Gevecht   
136265  6308709      Sharaputdin Magomedov vs Michael Page  Winnaar Gevecht   
136266  6308709      Sharaputdin Magomedov vs Michael Page  Winnaar Gevecht   

                             Outcome Name  Odds (De

In [87]:
set(final_df[final_df['sport'].str.contains('Voetbal')].competition)

{'Argentinië Superliga',
 'Australië A-League',
 'Australië W-League',
 'Azerbeidzjan Premier League',
 'België 1B Pro League',
 'België Jupiler Pro League',
 'Brazil Campeonato Paranaense',
 'Brazil Gaucho A1',
 'Brazil Supercup',
 'Brazilië Mineiro Modulo I',
 'Brazilië Paulista A1\n',
 'Colombia Categoría Primera A',
 'Cyprus Beker',
 'Cyprus First Division',
 'Egypte Premier League',
 'Engeland Championship',
 'Engeland FA Super League Vrouwen',
 'Engeland League 2',
 'Engeland National League',
 'Engeland National League Noord',
 'Engeland National League Zuid',
 "England FA Women's Premier League Cup",
 'Finland Liiga Cup',
 'Frankrijk Beker',
 'Frankrijk Ligue 1',
 'Frankrijk Ligue 2',
 'Frankrijk Ligue National',
 'Guatemala Liga Nacional',
 'India I-League',
 'India Super League',
 'Indonesië Liga 1',
 'Italië Coppa Italia Serie C',
 'Italië Serie A',
 'Italië Serie B',
 'Italy Serie C',
 'Jamaica National Premier League',
 'Kroatië 1. HNL',
 'Mexico League MX Vrouwen',
 'Mexi

In [88]:
final_df[(final_df['Event Name'].str.contains('Milan')) & final_df['sport']=='Voetbal']

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition


In [89]:
[i for i in list(set(final_df['Market Name'])) if 'No Bet' in i]

['3e Periode - Draw No Bet',
 '1e Helft: Draw No Bet',
 'Draw No Bet',
 '2e Periode - Draw No Bet',
 '2e Helft: Draw No Bet',
 '1e Periode - Draw No Bet']

In [90]:
final_output_name = 'totoAllSports' + start_time + '.csv'
final_df.to_csv(final_output_name)

## Check id ranges

In [55]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store all match data
all_matches = []

# Initialize a dictionary to store sports and competitions by drillDownTagIds range
range_summary = {}

# Loop over drillDownTagIds ranges in steps of 1000
for start in range(0, 22000, 200):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        "startTimeFrom=2025-01-15T23:00:00Z&"
        "startTimeTo=2025-01-30T22:59:59Z&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "categoryCode=tennis&"  # Specify the desired category
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        sports_set = set()
        competitions_set = set()

        for event in data.get('data', {}).get('events', []):
            # Collect match data
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Collect sports and competitions
            sports_set.add(match["sport"])
            competitions_set.add(match["competition"])

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)

            matches.append(match)

        # Add matches to the all_matches list
        all_matches.extend(matches)

        # Add sports and competitions to the range summary
        range_summary[f"{start}-{start + 200}"] = {
            "sports": list(sports_set),
            "competitions": list(competitions_set)
        }

        # Sleep for a random duration between 1 and 2 seconds
        time.sleep(random.uniform(1, 2.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 200}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the range summary
for range_key, summary in range_summary.items():
    print(f"\nDrillDownTagIds Range: {range_key}")
    print(f"Sports: {summary['sports']}")
    print(f"Competitions: {summary['competitions']}")

# Save the match data to a CSV or Excel file if needed
final_matches_df.to_csv("matches_output.csv", index=False)

# Optionally, save the range summary to a file
pd.DataFrame.from_dict(range_summary, orient="index").to_csv("range_summary.csv")

An error occurred for drilldown ID range 0–200: 'NoneType' object is not iterable

DrillDownTagIds Range: 200-400
Sports: []
Competitions: []

DrillDownTagIds Range: 400-600
Sports: ['Rugby Union', 'Volleybal', 'Voetbal', 'Boksen']
Competitions: ['Frankrijk Pro D2', 'Internationale Gevechten', 'Turkije 1. Lig', 'Engeland Premiership', 'Finland Mestaruusliiga Vrouwen', 'Brazilië Superliga Mannen', 'CEV Cup Vrouwen', 'Polen PlusLiga', 'Brazilië Superliga Vrouwen', 'Kroatië 1. HNL', 'Duitsland 3.Liga', 'Griekenland Super League', 'United Rugby Championship', 'Portugal Primeira Liga', 'Portugal Liga 3', 'Duitsland Bundesliga', 'Spanje LaLiga', 'Polen 1. Liga', "Romania Men's Divizia A1", 'Engeland Premier League', 'Champions League Vrouwen', 'Zuid-Korea V-League Vrouwen', 'Duitsland 2.Bundesliga', 'Costa Rica Primera Division', 'CEV Challenge Cup Vrouwen', 'Engeland League 1', 'Eredivisie Dames', 'Eredivisie Heren', 'Duitsland DVL 1. Bundesliga', 'Turkije Süper Lig', 'Portugal Segunda Liga

In [58]:
final_matches_df[(final_matches_df['sport'] == 'Voetbal') & (final_matches_df['competition'].str.contains('Champions League'))]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
1121,6248669,Benfica vs Barcelona,2025-01-21T20:00:00Z,Benfica,Barcelona,UEFA Champions League,Europees,Voetbal,3.10,2.20
1122,6248641,FK Crvena Zvezda vs PSV,2025-01-21T20:00:00Z,FK Crvena Zvezda,PSV,UEFA Champions League,Europees,Voetbal,4.25,1.74
1123,6248636,Liverpool vs LOSC Lille,2025-01-21T20:00:00Z,Liverpool,LOSC Lille,UEFA Champions League,Europees,Voetbal,1.30,10.50
1124,6248642,Club Brugge vs Juventus,2025-01-21T20:00:00Z,Club Brugge,Juventus,UEFA Champions League,Europees,Voetbal,3.40,2.31
1125,6248637,Atlético Madrid vs Bayer Leverkusen,2025-01-21T20:00:00Z,Atlético Madrid,Bayer Leverkusen,UEFA Champions League,Europees,Voetbal,2.39,3.20
1126,6248632,Bologna vs Borussia Dortmund,2025-01-21T20:00:00Z,Bologna,Borussia Dortmund,UEFA Champions League,Europees,Voetbal,3.25,2.29
1127,6248668,Slovan Bratislava vs VfB Stuttgart,2025-01-21T20:00:00Z,Slovan Bratislava,VfB Stuttgart,UEFA Champions League,Europees,Voetbal,11.00,1.25
1128,6248657,Feyenoord vs Bayern München,2025-01-22T20:00:00Z,Feyenoord,Bayern München,UEFA Champions League,Europees,Voetbal,8.00,1.39
1129,6248646,AC Sparta Praag vs Inter Milan,2025-01-22T20:00:00Z,AC Sparta Praag,Inter Milan,UEFA Champions League,Europees,Voetbal,8.00,1.41
1130,6248650,AC Milan vs Girona,2025-01-22T20:00:00Z,AC Milan,Girona,UEFA Champions League,Europees,Voetbal,1.48,6.75


In [31]:
# range starts at 1500 and stops at 13000

In [32]:
# range_summary

In [33]:
# keys_with_basketbal = [key for key, value in range_summary.items() if 'Voetbal' in value.get('sports', [])]
# keys_with_basketbal

In [34]:
# keys_with_voetbal = [key for key, value in range_summary.items() if 'Basketbal' in value.get('sports', [])]
# keys_with_voetbal